In [ ]:
from sympy import symbols, cos, sin, diff, Function, simplify

# Define symbols
theta = symbols('theta')
k, e, h, GM, sigma = symbols('k e h GM sigma')

# Define r(theta)
r = (h ** 2 / GM) / (1 + e * cos(theta *sigma))

# Calculate dr/dtheta
dr_dtheta = diff(r, theta)

# Define omega(theta) = dtheta/dt
omega = h / r**2

# Calculate d^2r/dtheta^2
d2r_dtheta2 = diff(dr_dtheta, theta)

# Calculate domega/dtheta
domega_dtheta = diff(omega, theta)

dr_dt = diff(r,theta)*omega
# Calculate d^2r/dt^2 using chain rule
d2r_dt2 = simplify(d2r_dtheta2 * omega**2 + dr_dtheta * domega_dtheta * omega)

# Calculate d^2theta/dt^2
# Differentiate omega(theta)
d2theta_dt2 = simplify(domega_dtheta * omega)

# Display the results

eq1 = f"r= {r}".replace("cos", "np.cos").replace("sin", "np.sin")
eq2 = f"theta_dot= h/r**2".replace("cos", "np.cos").replace("sin", "np.sin")
eq3 = f"dr_dtheta = {dr_dtheta}".replace("cos", "np.cos").replace("sin", "np.sin")
eq4 = f"d2r_dtheta2 = {d2r_dtheta2}".replace("cos", "np.cos").replace("sin", "np.sin")
eq5 = f"domega_dtheta = {domega_dtheta}".replace("cos", "np.cos").replace("sin", "np.sin")
eq6 = f"d2r_dt2 = {d2r_dt2}".replace("cos", "np.cos").replace("sin", "np.sin")
eq7 = f"d2theta_dt2 = {d2theta_dt2}".replace("cos", "np.cos").replace("sin", "np.sin")
eq8 = f"dr_dt = {dr_dt}".replace("cos", "np.cos").replace("sin", "np.sin")

print(eq1)
print(eq2)
# print(eq3)
# print(eq4)
# print(eq5)
print(eq6)
# print(eq7)
print(eq8)


In [ ]:
# import numpy as np
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad
import matplotlib.pyplot as plt
from astropy import constants as cc, units as uu
from scipy.optimize import curve_fit

# Constants
G = cc.G  # gravitational constant in m^3 kg^-1 s^-2
M = cc.M_sun  # mass of the Sun in kg
c = cc.c.si.value  # speed of light in m/s
GM = (G*M).si.value # gravitational parameter for the Sun in m^3/s^2

def r_derivatives(theta, h, e, GM, x):
    sigma = x[0]
    # Expression for r, r_dot, and r_double_dot
    r= h**2/(GM*(e*np.cos(sigma*theta) + 1))
    theta_dot= h/r**2
    dr_dtheta = e*h**2*sigma*np.sin(sigma*theta)/(GM*(e*np.cos(sigma*theta) + 1)**2)
    d2r_dt2 = GM**3*e*sigma**2*(e*np.cos(sigma*theta) + 1)**2*np.cos(sigma*theta)/h**4
    dr_dt = GM*e*sigma*np.sin(sigma*theta)/h
    return r, dr_dt, d2r_dt2, theta_dot


def error_functionHU(x, h, e, GM):
    sigma = x[0]
    def integrand(theta):
        r, r_dot, r_double_dot,theta_dot = r_derivatives(theta, h, e, GM, x)
        a_theoretical = HU_Accel(GM, r, r_dot, theta_dot)
        a_numerical = r_double_dot - r * theta_dot ** 2  # Assuming r_double_dot is defined 
                                                         # to calculate \ddot{r}
        return (a_theoretical - a_numerical)**2

    sigma = x[0]
    integral_error, _ = quad(integrand, 0, 2*np.pi/sigma, epsabs=1e-16, epsrel=1e-16)
    return 100*integral_error


def HU_Accel(GM, r, r_dot, theta_dot):
    v1 = np.sqrt(r_dot**2 + r**2 * theta_dot**2)
    gamma_v1 = 1 / np.sqrt(1 - v1**2 / c**2)
    P2_hat = np.sqrt(1 + v1**2/c**2 - 2*r_dot/c)
    a_theoretical = -GM /r**2*(P2_hat * gamma_v1 **(-3)/( (gamma_v1 - 1)*(r_dot/v1)**2 +1 ) )
    return a_theoretical

def calculatePrecession(errorf, a, e, T):
    h = np.sqrt(GM * a * (1 - e ** 2))
    # Initial guess for sigma
    initial_x = [0.95]
    result = minimize(errorf, initial_x, method='Nelder-Mead', args=(h,e,GM) ,tol=1E-15)
    optimized_sigma = result.x  # The optimized value for sigma
    if( not result.success):
        print(result)
    days_in_century = 36525  # Number of days in a century

    # Calculation of precession per orbit in radians
    delta_phi_per_orbit = 2 * np.pi * (optimized_sigma-1)
    delta_phi_per_orbit_GR = (6 * np.pi * GM) / (a * (1 - e ** 2) * c ** 2)

    # Calculate the number of orbits per century
    orbits_per_century = days_in_century / T

    # Precession per century in arcseconds
    delta_phi_per_century = delta_phi_per_orbit * orbits_per_century * (
                180 / np.pi) * 3600  # Convert radians to arcseconds
    delta_phi_per_century_GR = delta_phi_per_orbit_GR * orbits_per_century * (
                180 / np.pi) * 3600  # Convert radians to arcseconds


    return delta_phi_per_century[0], delta_phi_per_century_GR, optimized_sigma[0]-1


# https://nssdc.gsfc.nasa.gov/planetary/factsheet/mercuryfact.html
# https://nssdc.gsfc.nasa.gov/planetary/factsheet/venusfact.html
# https://nssdc.gsfc.nasa.gov/planetary/factsheet/marsfact.html
# https://nssdc.gsfc.nasa.gov/planetary/factsheet/jupiterfact.html
# https://nssdc.gsfc.nasa.gov/planetary/factsheet/saturnfact.html
# https://nssdc.gsfc.nasa.gov/planetary/factsheet/uranusfact.html
# https://nssdc.gsfc.nasa.gov/planetary/factsheet/neptunefact.html

# https://farside.ph.utexas.edu/teaching/336k/Newton/node115.html#e13.126

a_ = [57.909E9, 108.210E9, 149.598E9, 227.956E9, 778.479E9, 1432.041E9,  2867.043E9,   4514.953E9]
e_ = [0.2056,   0.0068,    0.0167,    0.0935,    0.0487,    0.0520,       0.0469,      0.0097]
T_ = [87.969,   224.701,   365.256,   686.980,   4332.589, 10759.22,    30685.4,      60189]


Planet = ["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]

df = pd.DataFrame(index=Planet, columns=['Planet', 'delta_HU', "delta_GR"])
df["Planet"] = Planet
for i, (a, e, T, name) in enumerate(zip(a_, e_, T_, Planet)):
    df.loc[name, ['delta_HU', "delta_GR", "sigma_HU"]] = calculatePrecession(error_functionHU, a, e, T)


ax = df.plot.scatter(x="Planet", y="delta_GR", label='GR Data', color='red')
df.plot(x="Planet", y="delta_HU", label='HU Data', color='blue', ax=ax)

ax.set_ylabel('Perihelion Precession (arcseconds/century)')
ax.legend()
# ax.set_yscale("log")

plt.tight_layout()
plt.savefig("./Drawing_For_Publications/HU_GR_Comparison.png")
plt.show()

In [ ]:
df.sigma_HU

In [ ]:
# Observed precession rates (arcseconds/year)
delta_Psi_obs = [5.75, 2.04, 11.45, 16.28, 6.55, 19.50, 3.34, 0.36]

# Theoretical precession rates (arcseconds/year)
delta_Psi_th = [5.50, 10.75, 11.87, 17.60, 7.42, 18.36, 2.72, 0.65]

# Convert to precession rates per century
df [ "delta_Psi_obs_century" ] = [rate * 100 for rate in delta_Psi_obs]
df [ "delta_Psi_th_century" ] = [rate * 100 for rate in delta_Psi_th]


df["Planet"] = ["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]
df["delta_obs"] = [43.11, 8.40, 3.84, 1.35, 0.06, 0.013, 0.0023, 0.0008]  # Observed values
df["error_obs"] = [0.21, 0.15, 0.11, 0.05, 0.02, 0.001, 0.0002, 0.0001]   # Error bars for observed values
df

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


df.set_index('Planet', inplace=True)

# Plot
fig, ax = plt.subplots()

# Plot delta_HU and delta_GR
df[['delta_HU', 'delta_GR']].plot(kind='bar', ax=ax)

# Add error bars for delta_GR
ax.errorbar(df.index, df['delta_GR'], yerr=df['error_obs'], fmt='o', color='black', capsize=5)

# Labels and title
ax.set_ylabel('Precession rate (arcseconds/century)')
ax.set_title('Comparison of Precession Rates: HU vs GR')
ax.legend(['HU', 'GR'])
ax.set_yscale("log")

plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Data from the document and your calculations
data = {
    "Planet": ["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"],
    "delta_HU": [38.589497, 8.274852, 3.694609, 1.277684, 0.059647, 0.013001, 0.002325, 0.000693],
    "delta_GR": [42.964879, 8.625333, 3.838716, 1.35111, 0.062313, 0.01364, 0.002386, 0.000775],
    "delta_obs": [43.11, 8.40, 3.84, 1.35, 0.06, 0.013, 0.0023, 0.0008],  # Observed values
    "error_obs": [0.21, 0.15, 0.11, 0.05, 0.02, 0.001, 0.0002, 0.0001]   # Error bars for observed values
}

# Create DataFrame
df = pd.DataFrame(data)
df.set_index('Planet', inplace=True)

# Plot
fig, ax = plt.subplots()

# Plot delta_HU, delta_GR, and delta_obs
df[['delta_HU', 'delta_GR', 'delta_obs']].plot(kind='bar', ax=ax)

# Add error bars for observations
ax.errorbar(df.index, df['delta_obs'], yerr=df['error_obs'], fmt='o', color='black', capsize=5)

# Labels and title
ax.set_ylabel('Precession rate (arcseconds/century)')
ax.set_title('Comparison of Precession Rates: HU vs GR vs Observations')
ax.legend(['HU', 'GR', 'Observations'])

plt.xticks(rotation=45)
ax.set_yscale("log")
plt.tight_layout()
plt.savefig("./Drawing_For_Publications/HU_GR_Obs_Comparison.png")
plt.show()


In [ ]:
import pandas as pd

# Data
a_ = [57.909E9, 108.210E9, 149.598E9, 227.956E9, 778.479E9, 1432.041E9, 2867.043E9, 4514.953E9]
e_ = [0.2056, 0.0068, 0.0167, 0.0935, 0.0487, 0.0520, 0.0469, 0.0097]
T_ = [87.969, 224.701, 365.256, 686.980, 4332.589, 10759.22, 30685.4, 60189]

# Observed precession rates (arcseconds/year)
delta_Psi_obs = [5.75, 2.04, 11.45, 16.28, 6.55, 19.50, 3.34, 0.36]

# Theoretical precession rates (arcseconds/year)
delta_Psi_th = [5.50, 10.75, 11.87, 17.60, 7.42, 18.36, 2.72, 0.65]

# Convert to precession rates per century
delta_Psi_obs_century = [rate * 100 for rate in delta_Psi_obs]
delta_Psi_th_century = [rate * 100 for rate in delta_Psi_th]

# Create DataFrame
df = pd.DataFrame({
    "planet": ["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"],
    "semi_major_axis": a_,
    "eccentricity": e_,
    "period": T_,
    "delta_Psi_obs_year": delta_Psi_obs,
    "delta_Psi_obs_century": delta_Psi_obs_century,
    "delta_Psi_th_year": delta_Psi_th,
    "delta_Psi_th_century": delta_Psi_th_century
})

print(df)


In [ ]:
import pandas as pd

# Data
a_ = [57.909E9, 108.210E9, 149.598E9, 227.956E9, 778.479E9, 1432.041E9, 2867.043E9, 4514.953E9]
e_ = [0.2056, 0.0068, 0.0167, 0.0935, 0.0487, 0.0520, 0.0469, 0.0097]
T_ = [87.969, 224.701, 365.256, 686.980, 4332.589, 10759.22, 30685.4, 60189]

# Observed precession rates (arcseconds/year)
delta_Psi_obs = [5.75, 2.04, 11.45, 16.28, 6.55, 19.50, 3.34, 0.36]

# Theoretical precession rates (arcseconds/year)
delta_Psi_th = [5.50, 10.75, 11.87, 17.60, 7.42, 18.36, 2.72, 0.65]

# Convert to precession rates per century
delta_Psi_obs_century = [rate * 100 for rate in delta_Psi_obs]
delta_Psi_th_century = [rate * 100 for rate in delta_Psi_th]

# Calculate relativistic contribution
delta_Psi_rel = [obs - th for obs, th in zip(delta_Psi_obs_century, delta_Psi_th_century)]

# Create DataFrame
df = pd.DataFrame({
    "planet": ["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"],
    "semi_major_axis": a_,
    "eccentricity": e_,
    "period": T_,
    "delta_Psi_obs_year": delta_Psi_obs,
    "delta_Psi_obs_century": delta_Psi_obs_century,
    "delta_Psi_th_year": delta_Psi_th,
    "delta_Psi_th_century": delta_Psi_th_century,
    "delta_Psi_rel": delta_Psi_rel
})

print(df)


In [ ]:
# Import necessary libraries
import numpy as np
import scipy.integrate
import matplotlib.pyplot as plt

# Define physical constants manually (avoiding astropy dependencies)
G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
M_sun = 1.9885e30  # Mass of the Sun in kg
c_si = 299792458  # Speed of light in m/s

# Compute Sun's gravitational parameter
GM_sun = G * M_sun  # m^3/s^2

# Mercury orbital parameters
a_mercury = 57.909e9  # Semi-major axis in meters
e_mercury = 0.2056  # Eccentricity
T_mercury = 87.969 * 24 * 3600  # Orbital period in seconds

# Define planetary parameters (semi-major axis in meters, eccentricity, orbital period in seconds)
planets = {
    "Venus": (108.210e9, 0.0068, 224.701 * 24 * 3600),
    "Earth": (149.598e9, 0.0167, 365.256 * 24 * 3600),
    "Mars": (227.956e9, 0.0935, 686.980 * 24 * 3600),
    "Jupiter": (778.479e9, 0.0487, 4332.589 * 24 * 3600),
    "Saturn": (1432.041e9, 0.0520, 10759.22 * 24 * 3600),
    "Uranus": (2867.043e9, 0.0469, 30685.4 * 24 * 3600),
    "Neptune": (4514.953e9, 0.0097, 60189 * 24 * 3600)
}

# Define planetary masses (kg) and compute GM_p values
planetary_masses = {
    "Venus": 4.8675e24,
    "Earth": 5.9724e24,
    "Mars": 6.4171e23,
    "Jupiter": 1.8982e27,
    "Saturn": 5.6834e26,
    "Uranus": 8.6810e25,
    "Neptune": 1.0241e26
}

# Compute GM_p for each planet
GM_planets = {planet: G * mass for planet, mass in planetary_masses.items()}

# Use correct sigma value
planetary_delta_sigma = {
    "Mercury": 7.214840e-08,
    "Venus": 3.913145e-08,
    "Earth": 2.859626e-08,
    "Mars": 1.859989e-08,
    "Jupiter": 5.498500e-09,
    "Saturn": 3.015393e-09,
    "Uranus": 1.423187e-09,
    "Neptune": 1.019615e-09,
}

sigma = 1 + planetary_delta_sigma["Mercury"]  # Use correct sigma value

# Define HU Acceleration function with numerical safeguards
def HU_Accel(GM, r, r_dot, theta_dot):
    v1 = np.sqrt(r_dot**2 + r**2 * theta_dot**2)
    v1 = min(v1, 0.99 * c_si)  # Avoid unphysical values near c
    
    gamma_v1 = 1 / np.sqrt(1 - v1**2 / c_si**2)
    P2_hat = np.sqrt(max(1 + v1**2 / c_si**2 - 2 * r_dot / c_si, 1e-10))  # Avoid negative sqrt
    
    a_theoretical = -GM / r**2 * (P2_hat * gamma_v1 ** (-3) / ((gamma_v1 - 1) * (r_dot / v1) ** 2 + 1))
    return a_theoretical

# Define equations of motion for Mercury including planetary perturbations
def equations_of_motion(t, y):
    r, theta, r_dot, theta_dot = y

    # Compute central force from Sun using HU acceleration
    a_r = HU_Accel(GM_sun, r, r_dot, theta_dot)
    a_theta = 0  # Initialize tangential acceleration

    # Compute perturbation forces from planets
    for planet, (a_p, e_p, T_p) in planets.items():
        GM_p = GM_planets[planet]  # Get correct GM for each planet

        # Approximate planetary motion as elliptical
        r_p = a_p * (1 - e_p * np.cos(2 * np.pi * t / T_p))
        theta_p = 2 * np.pi * t / T_p

        # Compute distance between Mercury and the planet
        delta_r = np.sqrt(r**2 + r_p**2 - 2 * r * r_p * np.cos(theta - theta_p))
        
        # Compute force magnitude from planet on Mercury
        F_p = GM_p / delta_r**2

        # Resolve force into radial and tangential components
        F_p_r = F_p * (r_p - r * np.cos(theta - theta_p)) / delta_r  # Radial component
        F_p_theta = F_p * (r * np.sin(theta - theta_p)) / delta_r  # Tangential component

        # Apply corrections to radial and tangential accelerations
        a_r += F_p_r
        a_theta += F_p_theta / r

    # Convert accelerations to velocity derivatives
    r_ddot = r * theta_dot**2 + a_r
    theta_ddot = -2 * r_dot * theta_dot / r + a_theta

    return [r_dot, theta_dot, r_ddot, theta_ddot]

# Initial conditions
r0 = a_mercury * (1 - e_mercury)  # Perihelion distance
theta0 = 0  # Start at perihelion
r_dot0 = 0  # Radial velocity at perihelion
theta_dot0 = np.sqrt(GM_sun * (1 + e_mercury) / (a_mercury * (1 - e_mercury)))  # Angular velocity

# Integration time (1 full Mercury orbit)
t_span = (0, T_mercury)
y0 = [r0, theta0, r_dot0, theta_dot0]

# Solve the differential equations
sol = scipy.integrate.solve_ivp(equations_of_motion, t_span, y0, method='RK45', t_eval=np.linspace(0, T_mercury, 1000))

# Extract results
r_sol, theta_sol = sol.y[0], sol.y[1]

# Convert to Cartesian coordinates for plotting
x_sol = r_sol * np.cos(theta_sol)
y_sol = r_sol * np.sin(theta_sol)

# Plot Mercury's orbit with precession
plt.figure(figsize=(8, 8))
plt.plot(x_sol, y_sol, label="Mercury Orbit (HU, Fixed)")
plt.scatter([0], [0], color='yellow', marker='o', label="Sun")
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.title("Mercury Orbit with HU Acceleration (Fixed)")
plt.legend()
plt.grid()
plt.axis("equal")
plt.show()

# Compute perihelion precession
perihelion_angles_fixed = np.unwrap(np.arctan2(y_sol, x_sol))  # Compute angles in radians
perihelion_shifts_fixed = np.diff(perihelion_angles_fixed) * (180 / np.pi) * 3600  # Convert to arcseconds

# Total precession per orbit (arcseconds)
total_precession_fixed = np.sum(perihelion_shifts_fixed)
total_precession_per_century_fixed = total_precession_fixed * (100 / (T_mercury / (365.25 * 24 * 3600)))

# Display corrected precession results
total_precession_per_century_fixed


In [ ]:
# Redefining constants manually instead of using astropy
G = 6.67430e-11  # Gravitational constant in m^3/kg/s^2
M_sun = 1.989e30  # Mass of the Sun in kg
c_si = 299792458  # Speed of light in m/s

# Compute Sun's GM manually
GM_sun = G * M_sun  # m^3/s^2

# Re-run the calculations with manually defined constants
sol = scipy.integrate.solve_ivp(equations_of_motion, t_span, y0, method='RK45', t_eval=np.linspace(0, T_mercury, 1000))

# Extract results
r_sol, theta_sol = sol.y[0], sol.y[1]

# Convert to Cartesian coordinates for plotting
x_sol = r_sol * np.cos(theta_sol)
y_sol = r_sol * np.sin(theta_sol)

# Plot Mercury's orbit with precession
plt.figure(figsize=(8, 8))
plt.plot(x_sol, y_sol, label="Mercury Orbit (HU, Fixed)")
plt.scatter([0], [0], color='yellow', marker='o', label="Sun")
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.title("Mercury Orbit with HU Acceleration (Fixed)")
plt.legend()
plt.grid()
plt.axis("equal")
plt.show()

# Compute perihelion precession
perihelion_angles = np.unwrap(np.arctan2(y_sol, x_sol))  # Compute angles in radians
perihelion_shifts = np.diff(perihelion_angles) * (180 / np.pi) * 3600  # Convert to arcseconds

# Total precession per orbit (arcseconds)
total_precession = np.sum(perihelion_shifts)
total_precession_per_century = total_precession * (100 / (T_mercury / (365.25 * 24 * 3600)))

# Display precession results
total_precession_per_century


In [ ]:
# Redefining constants manually instead of using astropy
G = 6.67430e-11  # Gravitational constant in m^3/kg/s^2
M_sun = 1.989e30  # Mass of the Sun in kg
c_si = 299792458  # Speed of light in m/s

# Compute Sun's GM manually
GM_sun = G * M_sun  # m^3/s^2

# Re-run the calculations with manually defined constants
sol = scipy.integrate.solve_ivp(equations_of_motion, t_span, y0, method='RK45', t_eval=np.linspace(0, T_mercury, 1000))

# Extract results
r_sol, theta_sol = sol.y[0], sol.y[1]

# Convert to Cartesian coordinates for plotting
x_sol = r_sol * np.cos(theta_sol)
y_sol = r_sol * np.sin(theta_sol)

# Plot Mercury's orbit with precession
plt.figure(figsize=(8, 8))
plt.plot(x_sol, y_sol, label="Mercury Orbit (HU, Fixed)")
plt.scatter([0], [0], color='yellow', marker='o', label="Sun")
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.title("Mercury Orbit with HU Acceleration (Fixed)")
plt.legend()
plt.grid()
plt.axis("equal")
plt.show()

# Compute perihelion precession
perihelion_angles = np.unwrap(np.arctan2(y_sol, x_sol))  # Compute angles in radians
perihelion_shifts = np.diff(perihelion_angles) * (180 / np.pi) * 3600  # Convert to arcseconds

# Total precession per orbit (arcseconds)
total_precession = np.sum(perihelion_shifts)
total_precession_per_century = total_precession * (100 / (T_mercury / (365.25 * 24 * 3600)))

# Display precession results
total_precession_per_century
